In [30]:
### original Bert for rule interestingness classification

In [31]:
import sys

datasetName = 'ncvoter' #sys.argv[1] #'airports' # hospital, ncvoter, inspection
mlmOption = False #sys.argv[2]
result_file = 'test.txt' #sys.argv[3]
train_ratio = 0.6 #(float)(sys.argv[4])

import os
import pandas as pd
import numpy as np
import copy

model_checkpoint_ = "distilbert-base-uncased"
mlm_model_dir = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-mlm')
saved_model_path = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-finetune')

model_checkpoint = copy.deepcopy(model_checkpoint_)
if mlmOption == True:
    model_checkpoint = mlm_model_dir #"distilbert-base-uncased"
data_prefix = '../../REEs_model_data/revision/labeled_data/'
rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')

In [32]:
# process rules

data_prefix = '../../REEs_model_data/revision/labeled_data/'
rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')

rules_set = pd.read_csv(rules_set_path)
rules_set = rules_set.values

for option in ('train', 'valid', 'test'):
    rule_pair_set_path = os.path.join(data_prefix, datasetName, 'train', option + '.csv')
    rules_pair_ids_set = pd.read_csv(rule_pair_set_path, delimiter=",")
    #rules_pair_ids_set
    # split training, validation and testing
    # prepare the rule pairs
    rules_pairs_set = []
    rules_pair_ids_set = rules_pair_ids_set.values
    if option == 'train':
        np.random.seed(20)
        np.random.shuffle(rules_pair_ids_set)
        rules_pair_ids_set = rules_pair_ids_set[:int(len(rules_pair_ids_set) * train_ratio)]
    for left_id, right_id, label in rules_pair_ids_set:
        rule_left, rule_right = rules_set[left_id][0], rules_set[right_id][0]
        rules_pairs_set.append([rule_left, rule_right, label])
    rules_pairs_set = np.array(rules_pairs_set)
    #rules_pairs_set[:10]
    # save csv
    rules_pairs_df = pd.DataFrame(rules_pairs_set, columns=['rule_left', 'rule_right', 'label'])
    output_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
    rules_pairs_df.to_csv(output_file, index=False)

In [33]:
# set gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [34]:
import transformers

from datasets import load_metric

In [35]:
from datasets import load_dataset

#dataset = load_dataset('csv', data_files='./top-k/rules/test_rules.csv')
_datasets = []
for option in ('train', 'valid', 'test'):
    rule_pairs_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
    tmp_dataset = load_dataset('csv', data_files=rule_pairs_file)
    _datasets.append(tmp_dataset)

train_dataset, valid_dataset, test_dataset = _datasets

Using custom data configuration default-e40419f3dcbc1350


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/user/.cache/huggingface/datasets/csv/default-e40419f3dcbc1350/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


Using custom data configuration default-db7a43fa5f9f46f6


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/user/.cache/huggingface/datasets/csv/default-db7a43fa5f9f46f6/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


Using custom data configuration default-ca5a24981ae80192
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-ca5a24981ae80192/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23)


In [36]:
train_dataset['train']['rule_left']

['ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.city_id2 == t1.city_id2 ⋀ t0.election_phase == 05/06/2008 PRIMARY  ->  t0.election_phase == t1.election_phase',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.county_id == t1.county_id ⋀ t0.election_phase == 11/08/2016 GENERAL  ->  t0.election_phase == t1.election_phase',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.county_desc == t1.county_desc ⋀ t0.election_phase == 05/06/2008 PRIMARY  ->  t0.election_phase == t1.election_phase',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.date == 11/06/2012 ⋀ t0.election_phase == t1.election_phase  ->  t0.election_phase == 11/06/2012',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.date == 11/08/2016 ⋀ t0.party == t1.party  ->  t0.election_phase == 11/08/2016',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.city_id3 == t1.city_id3 ⋀ t0.county_id == t1.county_id ⋀ t0.election_phase == 11/08/2016 GENERAL  ->  t0.city == t1.city',
 'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.city_id2 == t1.city_id2 ⋀ t0.county_id == t1.county_id ⋀ t0.election_phase == t1.election_phase  ->  t0.city == t1.city',


In [37]:
dataset = train_dataset
dataset['validation'] = valid_dataset['train']
dataset['test'] = test_dataset['train']
# dataset['train'] = dataset['train']
dataset

DatasetDict({
    train: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 287
    })
    validation: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 159
    })
    test: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 161
    })
})

In [38]:
from transformers import AutoTokenizer
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_, use_fast=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /home/user/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.

In [39]:
rule1_key, rule2_key = 'rule_left', 'rule_right'

dataset['train'][0][rule1_key]

'ncvoter(t0) ⋀ ncvoter(t1) ⋀ t0.city_id2 == t1.city_id2 ⋀ t0.election_phase == 05/06/2008 PRIMARY  ->  t0.election_phase == t1.election_phase'

In [40]:
def preprocess_function(examples):
    return tokenizer(examples[rule1_key], examples[rule2_key], truncation=True)

In [41]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 13316, 22994, 2121, 1006, 1056, 2692, 1007, 100, 13316, 22994, 2121, 1006, 1056, 2487, 1007, 100, 1056, 2692, 1012, 2103, 1035, 8909, 2475, 1027, 1027, 1056, 2487, 1012, 2103, 1035, 8909, 2475, 100, 1056, 2692, 1012, 2602, 1035, 4403, 1027, 1027, 5709, 1013, 5757, 1013, 2263, 3078, 1011, 1028, 1056, 2692, 1012, 2602, 1035, 4403, 1027, 1027, 1056, 2487, 1012, 2602, 1035, 4403, 102, 13316, 22994, 2121, 1006, 1056, 2692, 1007, 100, 13316, 22994, 2121, 1006, 1056, 2487, 1007, 100, 1056, 2692, 1012, 2103, 1035, 8909, 2475, 1027, 1027, 1056, 2487, 1012, 2103, 1035, 8909, 2475, 100, 1056, 2692, 1012, 2221, 1035, 8909, 1027, 1027, 1056, 2487, 1012, 2221, 1035, 8909, 100, 1056, 2692, 1012, 2602, 1035, 4403, 1027, 1027, 2340, 1013, 6185, 1013, 2230, 2236, 100, 1056, 2692, 1012, 2126, 1035, 1997, 1035, 6830, 1027, 1027, 1056, 2487, 1012, 2126, 1035, 1997, 1035, 6830, 1011, 1028, 1056, 2692, 1012, 2103, 1027, 1027, 1056, 2487, 1012, 2103, 102], [101, 13316, 22994, 2121, 1006, 

In [42]:
#tokenizer.decode(_['input_ids'][0])

In [43]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-ca5a24981ae80192/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23/cache-8e6e44eab850c17a.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 287
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 159
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 161
    })
})

In [44]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 287
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 159
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'rule_left', 'rule_right'],
        num_rows: 161
    })
})

In [45]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /home/user/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637

In [46]:
metric = 'accuracy'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(output_dir=saved_model_path,
                        evaluation_strategy='epoch',
                        save_strategy='epoch',
                        learning_rate=2e-5,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        num_train_epochs=20,
                        weight_decay=0.01,
                        load_best_model_at_end=True,
                        metric_for_best_model='accuracy',
                        push_to_hub=False)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [47]:

# >>> metrics.precision_score(y_true, y_pred)
# 1.0
# >>> metrics.recall_score(y_true, y_pred)
# 0.5
# >>> metrics.f1_score(y_true, y_pred)
# accuracy_score(y_true, y_pred)
from sklearn import metrics
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #print(predictions, labels)
    precision = metrics.precision_score(labels, predictions)
    recall = metrics.recall_score(labels, predictions)
    f1 = metrics.f1_score(labels, predictions)
    acc = metrics.accuracy_score(labels, predictions)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': acc}


trainer = Trainer(model, args, train_dataset=encoded_dataset['train'], eval_dataset=encoded_dataset['validation'], tokenizer=tokenizer, compute_metrics=compute_metrics)


In [48]:

os.environ["WANDB_DISABLED"] = "true"

In [49]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running training *****
  Num examples = 287
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 360


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.686866,0.750000,0.862069,0.802139,0.767296
2,No log,0.544434,0.835616,0.701149,0.762500,0.761006
3,No log,0.410372,0.754545,0.954023,0.842640,0.805031
4,No log,0.429843,0.806452,0.862069,0.833333,0.811321
5,No log,0.370292,0.790476,0.954023,0.864583,0.836478
6,No log,0.358550,0.970149,0.747126,0.844156,0.849057
7,No log,0.335749,0.913580,0.850575,0.880952,0.874214
8,No log,0.345857,0.892857,0.862069,0.877193,0.867925
9,No log,0.412219,0.816327,0.919540,0.864865,0.842767
10,No log,0.401997,0.831579,0.908046,0.868132,0.849057


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16
Saving model checkpoint to ./saved_models/ncvoter-distilbert-base-uncased-finetune/checkpoint-18
Configuration saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/checkpoint-18/config.json
Model weights saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/checkpoint-18/pytorch_model.bin
tokenizer config file saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/checkpoint-18/tokenizer_config.json
Special tokens file saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/checkpoint-18/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evalu

TrainOutput(global_step=360, training_loss=0.22781490749782987, metrics={'train_runtime': 118.3245, 'train_samples_per_second': 48.511, 'train_steps_per_second': 3.042, 'total_flos': 244668320219184.0, 'train_loss': 0.22781490749782987, 'epoch': 20.0})

In [50]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16


{'eval_loss': 0.5147467255592346,
 'eval_precision': 0.8953488372093024,
 'eval_recall': 0.8850574712643678,
 'eval_f1': 0.8901734104046243,
 'eval_accuracy': 0.8805031446540881,
 'eval_runtime': 0.3153,
 'eval_samples_per_second': 504.22,
 'eval_steps_per_second': 31.712,
 'epoch': 20.0}

In [51]:
ll = trainer.evaluate(encoded_dataset['test'])
print(ll)

f = open(result_file, 'w')
f.write(str(ll))
f.close()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 161
  Batch size = 16


{'eval_loss': 0.35205912590026855, 'eval_precision': 0.8918918918918919, 'eval_recall': 0.8918918918918919, 'eval_f1': 0.8918918918918919, 'eval_accuracy': 0.9006211180124224, 'eval_runtime': 0.3332, 'eval_samples_per_second': 483.191, 'eval_steps_per_second': 33.013, 'epoch': 20.0}


In [52]:
# save model
trainer.save_model()

Saving model checkpoint to ./saved_models/ncvoter-distilbert-base-uncased-finetune
Configuration saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/config.json
Model weights saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/pytorch_model.bin
tokenizer config file saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/tokenizer_config.json
Special tokens file saved in ./saved_models/ncvoter-distilbert-base-uncased-finetune/special_tokens_map.json
